In [13]:
import pandas as pd
import geopandas as gpd
from unidecode import unidecode
from geopy.distance import great_circle


In [3]:
gdf=gpd.read_file('../data/raw/fr/communes-20190101.json')
gdf['centroid']=gdf.geometry.centroid
centroid_map = gdf.set_index('nom')['centroid'].to_dict()

<ipython-input-3-72af4f269941>:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroid']=gdf.geometry.centroid


In [4]:
mobilite=pd.read_csv('../data/raw/fr/base-flux-mobilite-domicile-lieu-travail-2017.csv', sep=';').rename(columns={'NBFLUX_C17_ACTOCC15P': 'TravellerCount'})

/home/niedakh/venvs/p3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3417: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
pop=pd.read_csv('../data/raw/fr/COMPOP.csv')
pop['COM_name']=pop.COM.apply(lambda x: '-'.join(x.split('-')[:-1]))
population_map = pop.set_index('COM_name')['PTOT'].to_dict()

In [8]:
mob_df = mobilite.groupby(['LIBGEO', 'L_DCLT'])['TravellerCount'].sum().reset_index()
mob_df['SourcePopulation']=mob_df.LIBGEO.apply(lambda x: population_map.get(unidecode(x)))
mob_df['TargetPopulation']=mob_df.L_DCLT.apply(lambda x: population_map.get(unidecode(x)))

In [10]:
location_names = set(mob_df.L_DCLT.tolist()+mob_df.LIBGEO.tolist())
the_intersection = set(gdf.nom.tolist()).intersection(location_names)
len(location_names), len(the_intersection)

(34347, 32503)

In [14]:
final_mob_df = mob_df.query('LIBGEO in @the_intersection').query('L_DCLT in @the_intersection')
final_mob_df['SourceCentroid']=final_mob_df.LIBGEO.apply(lambda x: centroid_map[x])
final_mob_df['TargetCentroid']=final_mob_df.L_DCLT.apply(lambda x: centroid_map[x])
final_mob_df['Distance']=final_mob_df.apply(lambda row: great_circle(tuple(row['SourceCentroid'].coords)[0], tuple(row['TargetCentroid'].coords)[0]).km, axis=1)

In [ ]:
final_mob_df.columns

In [17]:
final_mob_df.to_pickle('../data/processed/fr_matrix.p')